<a href="https://colab.research.google.com/github/xlopez-ml/DL-AMR/blob/master/Examples/DeepAMR_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0-Libraries

#1-Load Data and Model

#2-Training model with only external data 

#3-Test external data on DRIAMS-A pretrained model 

#4-Test external data applying transfer learning, freezing convolutional layers.

#5-Test external data applying transfer learning, retraining all layers.